In [79]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [80]:
seed = 1
batch_size = 64
test_batch_size = 1000
momentum = 0.9
epochs = 50
log_interval = 100
lr = 0.01
save_model = False
num_classes = 10
torch.backends.cudnn.benchmark=True


In [81]:
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
#        self.conv1 = nn.Conv2d(1, 20, 5, 1)
#        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(32*32*3, 600)
        self.fc2 = nn.Linear(600, 600)
        self.fc3 = nn.Linear(600, num_classes)
        self.dropout1 = nn.Dropout2d(0.2)
        self.dropout2 = nn.Dropout2d(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
#        x = F.max_pool2d(x, 2, 2)
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
#        x = F.max_pool2d(x, 2, 2)
#        x = x.view(-1, 4*4*50)
#        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
#        x = self.fc2(x)
        return F.relu(self.fc3(x))

In [82]:
def train(model, device, train_loader, optimizer, log_interval, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.view(-1, 32*32*3).to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [83]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.view(-1, 32*32*3).to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).sum().item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [84]:
use_cuda =  torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}

torch.manual_seed(seed)


train_dataset=datasets.CIFAR10('./data_c', train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
#                       transforms.Normalize((0.1307,), (0.3081,))
                       
                   ]))

test_dataset=datasets.CIFAR10('./data_c', train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
#                       transforms.Normalize((0.1307,), (0.3081,))
    ]))

image, label = train_dataset[0]
print(image.size())
print(label)

Files already downloaded and verified
Files already downloaded and verified
torch.Size([3, 32, 32])
6


In [85]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=test_batch_size, shuffle=False, num_workers=2)


In [86]:
for images, labels in train_loader:
    print(images.size())
    print(images[0].size())
    print(labels.size())
    
    break

torch.Size([64, 3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([64])


In [91]:
model = MLPNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=5e-4)

def run():
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, log_interval, epoch)
        test(model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"mlp.pt")

In [92]:
%%time

run()

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.306810
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.184817
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.005188
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.990805
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.929721
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.732579
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.820583
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.912813

Test set: Average loss: 0.0018, Accuracy: 3688/10000 (37%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.850905
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.766968
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.748026
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.674198
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.788921
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.694371
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.720683
Train Epoch: 2 [44800/50000 (90%)]	Loss: 1.664682

Test set: Average loss: 0.0017, Accuracy: 3799/10000 (38%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 1.838473
Train Epoch: 3 [6400/50000 (13%)]	Loss: 1.746


Test set: Average loss: 0.0014, Accuracy: 4913/10000 (49%)

Train Epoch: 19 [0/50000 (0%)]	Loss: 1.259978
Train Epoch: 19 [6400/50000 (13%)]	Loss: 1.578281
Train Epoch: 19 [12800/50000 (26%)]	Loss: 1.282249
Train Epoch: 19 [19200/50000 (38%)]	Loss: 1.185177
Train Epoch: 19 [25600/50000 (51%)]	Loss: 1.528658
Train Epoch: 19 [32000/50000 (64%)]	Loss: 1.539751
Train Epoch: 19 [38400/50000 (77%)]	Loss: 1.510812
Train Epoch: 19 [44800/50000 (90%)]	Loss: 1.564573

Test set: Average loss: 0.0014, Accuracy: 4970/10000 (50%)

Train Epoch: 20 [0/50000 (0%)]	Loss: 1.600177
Train Epoch: 20 [6400/50000 (13%)]	Loss: 1.632260
Train Epoch: 20 [12800/50000 (26%)]	Loss: 1.275313
Train Epoch: 20 [19200/50000 (38%)]	Loss: 1.437822
Train Epoch: 20 [25600/50000 (51%)]	Loss: 1.694466
Train Epoch: 20 [32000/50000 (64%)]	Loss: 1.328590
Train Epoch: 20 [38400/50000 (77%)]	Loss: 1.402916
Train Epoch: 20 [44800/50000 (90%)]	Loss: 1.431000

Test set: Average loss: 0.0014, Accuracy: 5232/10000 (52%)

Train Epoch: 

Train Epoch: 36 [38400/50000 (77%)]	Loss: 1.273261
Train Epoch: 36 [44800/50000 (90%)]	Loss: 1.319896

Test set: Average loss: 0.0014, Accuracy: 5143/10000 (51%)

Train Epoch: 37 [0/50000 (0%)]	Loss: 1.520139
Train Epoch: 37 [6400/50000 (13%)]	Loss: 1.193302
Train Epoch: 37 [12800/50000 (26%)]	Loss: 1.310717
Train Epoch: 37 [19200/50000 (38%)]	Loss: 1.085726
Train Epoch: 37 [25600/50000 (51%)]	Loss: 1.252359
Train Epoch: 37 [32000/50000 (64%)]	Loss: 1.037482
Train Epoch: 37 [38400/50000 (77%)]	Loss: 1.224532
Train Epoch: 37 [44800/50000 (90%)]	Loss: 1.092328

Test set: Average loss: 0.0013, Accuracy: 5254/10000 (53%)

Train Epoch: 38 [0/50000 (0%)]	Loss: 1.510054
Train Epoch: 38 [6400/50000 (13%)]	Loss: 1.290339
Train Epoch: 38 [12800/50000 (26%)]	Loss: 1.193570
Train Epoch: 38 [19200/50000 (38%)]	Loss: 1.257985
Train Epoch: 38 [25600/50000 (51%)]	Loss: 1.359784
Train Epoch: 38 [32000/50000 (64%)]	Loss: 1.418045
Train Epoch: 38 [38400/50000 (77%)]	Loss: 1.091381
Train Epoch: 38 [44800/